In [2]:
# Import Libraries
import cv2
import os


import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
from sklearn.metrics import classification_report, confusion_matrix

from keras.models import Sequential #for neural network models
from keras.layers import Input, Dense, Flatten,Conv2D, ZeroPadding2D, MaxPooling2D, Dropout
from keras.layers import Conv2DTranspose, SeparableConv2D, BatchNormalization, Reshape # Import hidden layer
from keras.preprocessing.image import ImageDataGenerator #Data augmentation and preprocessing
from keras.optimizers import Adam #For Optimizing the Neural Network
from keras.applications.vgg16 import VGG16 # VGG16 saved model from keras
from keras import layers
from keras.layers import Input
from keras.models import Model
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [0]:
#use your dir for the data
good_path = "/content/drive/My Drive/AI project/dataset/good/"
blurred_path = "/content/drive/My Drive/AI project/dataset/blurred_noise/"

In [0]:
#Hyperparameters
img_width = 64
img_height = 64
img_channel = 1
input_shape = (img_height,img_width,img_channel)
num_epochs = 1
batch_size = 16
latent_dim = 10

In [0]:
noise_datagen = ImageDataGenerator(rescale=1./225,validation_split=0.25,dtype='float32')
img_datagen = ImageDataGenerator(rescale=1. / 255,validation_split=0.25, dtype='float32')


In [6]:
train_good = img_datagen.flow_from_directory(good_path,target_size=(img_width,img_height),
                                             class_mode=None,color_mode= 'grayscale',
                                             batch_size=batch_size,subset='training')
test_good = img_datagen.flow_from_directory(good_path,target_size=(img_width,img_height),
                                             class_mode=None,color_mode= 'grayscale',
                                            batch_size=batch_size,subset='validation')
train_blurred = noise_datagen.flow_from_directory(blurred_path,target_size=(img_width,img_height),
                                             class_mode=None,color_mode= 'grayscale',
                                             batch_size=batch_size,subset='training')
test_blurred = noise_datagen.flow_from_directory(blurred_path,target_size=(img_width,img_height),
                                             class_mode=None,color_mode= 'grayscale',
                                             batch_size=batch_size,subset='validation')

Found 1872 images belonging to 1 classes.
Found 624 images belonging to 1 classes.
Found 2200 images belonging to 1 classes.
Found 733 images belonging to 1 classes.


In [7]:
train_good.image_shape

(64, 64, 1)

In [0]:
# create generators using zip() that will train orginal and noisy images 
train_generator = zip(train_good,train_blurred)
test_generator = zip(test_good,test_blurred)

In [9]:
#check dense unit before and after bottleneck layer depending on the img_width , img_height
unit = (img_width/16)**2*latent_dim
print(unit)

160.0


In [10]:
#input shape
input_img = Input(input_shape) # adapt this if using `channels_first` image data format

#Encoder part
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.20)(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.20)(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(latent_dim, (3,3), padding='same', name='Conv5')(x)

#check shape
shape_before_flattening = K.int_shape(encoded)

x = Flatten()(encoded)
encoded_to_latent = Dense(160, activation='relu')(x) #use unit here
#bottleneck layer
latent = Dense(latent_dim, activation='relu', name='latent_space')(encoded_to_latent)

x = Dense(160, activation='relu')(latent) #use unit here

#Reshape
x = Reshape(shape_before_flattening[1:])(x)

#Decoder part
x = Conv2DTranspose(16, (2,2),padding='same', activation='relu',strides=(2, 2))(x)
x = Conv2DTranspose(32, (2,2),padding='same', activation='relu',strides=(2, 2))(x)
x = Conv2DTranspose(64, (2,2),padding='same', activation='relu',strides=(2, 2))(x)
x = Conv2DTranspose(64,(2,2),padding='same', activation='relu',strides=(2, 2))(x)
decoded = Conv2D(img_channel,(3,3), activation='sigmoid', padding='same')(x)


autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
autoencoder.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________

In [11]:
hist_test = autoencoder.fit_generator(train_generator,steps_per_epoch=16,
                                      epochs=num_epochs,
                                      validation_data=test_generator,verbose=1,
                                      validation_steps=16,
                                      shuffle=True,
                                      )




Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


KeyboardInterrupt: ignored